In [1]:
import os
import SimpleITK as sitk
import pydicom as pydicom

from tqdm import tqdm

import subprocess

import pandas as pd

import ants
import antspynet

import shutil
import time


2024-10-10 11:42:57.148314: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 11:42:57.454357: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/share/sw/gcc-8.5.0/slurm-22.05.7/lib:/opt/share/libs/gcc-8.5.0/ucx-1.13.1/lib:/opt/share/libs/gcc-8.5.0/pmix-4.2.2/lib:/opt/share/libs/gcc-8.5.0/hwloc-2.9.0/lib:/usr/lib64:/usr/lib:/opt/share/libs/nvidia/cuda-12.0.0/lib64:/opt/share/libs/nvidia/cuda-12.0.0/lib64/stubs
2024-10-10 11:42:57.454381: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart d

# Gathering non-FS T2w Image Samples

In [2]:
DATA_DIR = r"/home/thulasiseetha/research/dataset/curated/BrainMR/t2w_nfs"
OUT_DIR = r"outputs"

pids = os.listdir(DATA_DIR)[:2]
src_img_files = [os.path.join(DATA_DIR, pid, "img.nii.gz") for pid in pids]

In [3]:
df = {"pid":[],"case":[], "desc":[], "time":[]}

# Case-1: Freesurfer (Brain Extraction) + ANTsX (Tissue Segmentation)

In [4]:
case = "case1"
desc = r"Freesurfer (Brain Extraction) + ANTsX (Tissue Segmentation)"

In [5]:
for src_img_file in tqdm(src_img_files):
    
    start_time = time.time()
    
    pid = src_img_file.split("/")[-2]
    out_dir = os.path.join(OUT_DIR, pid)
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    if not os.path.exists(os.path.join(out_dir, "cache", "freesurfer")):
        os.makedirs(os.path.join(out_dir,"cache","freesurfer"))
    if not os.path.exists(os.path.join(out_dir, "cache", case)):
        os.makedirs(os.path.join(out_dir, "cache", case))
        
    dest_img_file = os.path.join(out_dir, "img.nii.gz")
        
    shutil.copy(src_img_file, dest_img_file)
    
    img_file = os.path.join(out_dir, "img.nii.gz")
    brainMask_file = os.path.join(out_dir,"cache","freesurfer","brainMask.nii.gz")
    brainMaskedImg_file = os.path.join(out_dir, "cache","freesurfer","brainMaskedImg.nii.gz")

    if (not os.path.exists(brainMask_file)) or (not os.path.exists(brainMaskedImg_file)):
        
        subprocess.run(f"mri_synthstrip -i {img_file} -m {brainMask_file} -o {brainMaskedImg_file}", shell=True)

    
    tissueMask_prefix = os.path.join(out_dir, "cache", case, "tissue")
    subprocess.run(f"antsAtroposN4.sh -d 3 -a {img_file} -x {brainMask_file} -c 3 -o {tissueMask_prefix}", shell=True)
    
    src_mask_file = os.path.join(out_dir, "cache", case, "tissueSegmentation.nii.gz")
    dest_mask_file = os.path.join(out_dir, f"{case}.nii.gz")
    
    shutil.copy(src_mask_file, dest_mask_file)
        
    
    elapsed_time = time.time()-start_time
    
    df["pid"].append(pid)
    df["case"].append(case)
    df["desc"].append(desc)
    df["time"].append(elapsed_time)


  0%|                                                                                                                           | 0/2 [00:00<?, ?it/s]

Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: outputs/paz40/img.nii.gz
Processing frame (of 1): 1 done
Masked image saved to: outputs/paz40/cache/freesurfer/brainMaskedImg.nii.gz
Binary brain mask saved to: outputs/paz40/cache/freesurfer/brainMask.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip

Initializing with kmeans segmentation.
---------------------  Running antsAtroposN4.sh on fn04.eos.medooza.lan  ---------------------
BEGIN >>>>>>>>>>>>>>>>>>>>
ImageMath 3 outputs/paz40/cache/case1/tissueSegmentationPreprocessedAnatomical0.nii.gz TruncateImageIntensity outputs/paz40/img.nii.gz 0 0.995 256



    Using antsAtroposN4 with the following arguments:
      image dimension         = 3
      anatomical image        = outputs/paz40/img.nii.gz
      segmentation priors     = 
      output prefix           = outputs/paz40/cache/case1/tissue
      output image suffix     = nii.gz
      denoise images          = 0

    N4 parameters (segmentation):
      convergence             = [ 50x50x50x50,0.0000000001 ]
      shrink factor           = 2
      B-spline parameters     = [ 200 ]
      weight mask post. label = 

    Atropos parameters (segmentation):
       convergence            = [ 5,0.0 ]
       likelihood             = Gaussian
       prior weight           = 0.0
       posterior formulation  = Socrates[ 1 ]
       mrf                    = [ 0.1,1x1x1 ]
       Max N4->Atropos iters. = 15
       Max Atropos iters.     = 5
       use clock random seed  = 1
       icm                    = 1
       use-euclidean-distance = 0



END   <<<<<<<<<<<<<<<<<<<<


BEGIN >>>>>>>>>>>>>>>>>>>>
N4BiasFieldCorrection -d 3 -i outputs/paz40/cache/case1/tissueSegmentationPreprocessedAnatomical0.nii.gz -x outputs/paz40/cache/freesurfer/brainMask.nii.gz -s 2 -c [ 50x50x50x50,0.0000000001 ] -b [ 200 ] -o outputs/paz40/cache/case1/tissueSegmentation0N4.nii.gz --verbose 1

Running N4 for 3-dimensional images.

Original intensity range:  [1, 1566.04]
Specified spline distance: 200mm
  original image size:  [512, 512, 22]
  padded image size:  [854, 854, 62]
  number of control points:  [5, 5, 4]

Current level = 1
  Iteration 1 (of 50).   Current convergence value = 0.000111517 (threshold = 1e-10)
  Iteration 2 (of 50).   Current convergence value = 0.000111252 (threshold = 1e-10)
  Iteration 3 (of 50).   Current convergence value = 0.000110996 (threshold = 1e-10)
  Iteration 4 (of 50).   Current convergence value = 0.000110599 (threshold = 1e-10)
  Iteration 5 (of 50).   Current convergence value = 0.000110344 (threshold = 1e-10)

  Iteration 40 (of 50).   Current convergence value = 0.000220429 (threshold = 1e-10)
  Iteration 41 (of 50).   Current convergence value = 0.000219708 (threshold = 1e-10)
  Iteration 42 (of 50).   Current convergence value = 0.000218948 (threshold = 1e-10)
  Iteration 43 (of 50).   Current convergence value = 0.000218135 (threshold = 1e-10)
  Iteration 44 (of 50).   Current convergence value = 0.000217414 (threshold = 1e-10)
  Iteration 45 (of 50).   Current convergence value = 0.000216735 (threshold = 1e-10)
  Iteration 46 (of 50).   Current convergence value = 0.000216049 (threshold = 1e-10)
  Iteration 47 (of 50).   Current convergence value = 0.000215493 (threshold = 1e-10)
  Iteration 48 (of 50).   Current convergence value = 0.000214841 (threshold = 1e-10)
  Iteration 49 (of 50).   Current convergence value = 0.000214166 (threshold = 1e-10)
  Iteration 50 (of 50).   Current convergence value = 0.000213466 (threshold = 1e-10)
Current level = 3
  Iteration 1 (of 50).   Current con

...

Original intensity range:  [1, 2305.62]
Specified spline distance: 200mm
  original image size:  [512, 512, 27]
  padded image size:  [854, 854, 62]
  number of control points:  [5, 5, 4]

Current level = 1
  Iteration 1 (of 50).   Current convergence value = 8.24064e-05 (threshold = 1e-10)
  Iteration 2 (of 50).   Current convergence value = 8.22499e-05 (threshold = 1e-10)
  Iteration 3 (of 50).   Current convergence value = 8.209e-05 (threshold = 1e-10)
  Iteration 4 (of 50).   Current convergence value = 8.19183e-05 (threshold = 1e-10)
  Iteration 5 (of 50).   Current convergence value = 8.17944e-05 (threshold = 1e-10)
  Iteration 6 (of 50).   Current convergence value = 8.16389e-05 (threshold = 1e-10)
  Iteration 7 (of 50).   Current convergence value = 8.14587e-05 (threshold = 1e-10)
  Iteration 8 (of 50).   Current convergence value = 8.1318e-05 (threshold = 1e-10)
  Iteration 9 (of 50).   Current convergence value = 8.11958e-05 (threshold = 1e-10)
  Iteration 10 (of 50).   Curre

  Iteration 44 (of 50).   Current convergence value = 0.000190767 (threshold = 1e-10)
  Iteration 45 (of 50).   Current convergence value = 0.000189527 (threshold = 1e-10)
  Iteration 46 (of 50).   Current convergence value = 0.000188306 (threshold = 1e-10)
  Iteration 47 (of 50).   Current convergence value = 0.000187101 (threshold = 1e-10)
  Iteration 48 (of 50).   Current convergence value = 0.000185906 (threshold = 1e-10)
  Iteration 49 (of 50).   Current convergence value = 0.000184742 (threshold = 1e-10)
  Iteration 50 (of 50).   Current convergence value = 0.000183643 (threshold = 1e-10)
Current level = 3
  Iteration 1 (of 50).   Current convergence value = 0.000577077 (threshold = 1e-10)
  Iteration 2 (of 50).   Current convergence value = 0.0005704 (threshold = 1e-10)
  Iteration 3 (of 50).   Current convergence value = 0.000564027 (threshold = 1e-10)
  Iteration 4 (of 50).   Current convergence value = 0.000558224 (threshold = 1e-10)
  Iteration 5 (of 50).   Current convergen

  Iteration 40 (of 50).   Current convergence value = 0.000715612 (threshold = 1e-10)
  Iteration 41 (of 50).   Current convergence value = 0.000709554 (threshold = 1e-10)
  Iteration 42 (of 50).   Current convergence value = 0.000703439 (threshold = 1e-10)
  Iteration 43 (of 50).   Current convergence value = 0.000697303 (threshold = 1e-10)
  Iteration 44 (of 50).   Current convergence value = 0.000691061 (threshold = 1e-10)
  Iteration 45 (of 50).   Current convergence value = 0.000684703 (threshold = 1e-10)
  Iteration 46 (of 50).   Current convergence value = 0.00067845 (threshold = 1e-10)
  Iteration 47 (of 50).   Current convergence value = 0.000671984 (threshold = 1e-10)
  Iteration 48 (of 50).   Current convergence value = 0.000665681 (threshold = 1e-10)
  Iteration 49 (of 50).   Current convergence value = 0.000659154 (threshold = 1e-10)
  Iteration 50 (of 50).   Current convergence value = 0.000652729 (threshold = 1e-10)
   N4BiasFieldCorrectionImageFilter (0x18ac6d0)
     RT

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [04:46<00:00, 143.36s/it]

END   <<<<<<<<<<<<<<<<<<<<


BEGIN >>>>>>>>>>>>>>>>>>>>
cp -f outputs/paz9/cache/case1/tissueSegmentationPosteriorsPreviousIteration3.nii.gz outputs/paz9/cache/case1/tissueSegmentationPosteriors3.nii.gz
END   <<<<<<<<<<<<<<<<<<<<


BEGIN >>>>>>>>>>>>>>>>>>>>
rm outputs/paz9/cache/case1/tissueSegmentationPosteriorsPreviousIteration1.nii.gz
END   <<<<<<<<<<<<<<<<<<<<


BEGIN >>>>>>>>>>>>>>>>>>>>
rm outputs/paz9/cache/case1/tissueSegmentationPosteriorsPreviousIteration2.nii.gz
END   <<<<<<<<<<<<<<<<<<<<


BEGIN >>>>>>>>>>>>>>>>>>>>
rm outputs/paz9/cache/case1/tissueSegmentationPosteriorsPreviousIteration3.nii.gz
END   <<<<<<<<<<<<<<<<<<<<


BEGIN >>>>>>>>>>>>>>>>>>>>
rm outputs/paz9/cache/case1/tissueSegmentationPreviousIteration.nii.gz
END   <<<<<<<<<<<<<<<<<<<<


BEGIN >>>>>>>>>>>>>>>>>>>>
rm outputs/paz9/cache/case1/tissueSegmentationPreprocessedAnatomical0.nii.gz
END   <<<<<<<<<<<<<<<<<<<<



--------------------------------------------------------------------------------------
 Done 

# Case-2: Freesurfer (Brain Extraction) + FAST (Tissue Segmentation)

In [6]:
case = "case2"
desc = r"Freesurfer (Brain Extraction) + FSL-FAST (Tissue Segmentation)"

In [8]:
for src_img_file in tqdm(src_img_files):
    
    start_time = time.time()
    
    pid = src_img_file.split("/")[-2]
    out_dir = os.path.join(OUT_DIR, pid)
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    if not os.path.exists(os.path.join(out_dir, "cache", "freesurfer")):
        os.makedirs(os.path.join(out_dir,"cache","freesurfer"))
    if not os.path.exists(os.path.join(out_dir, "cache", case)):
        os.makedirs(os.path.join(out_dir, "cache", case))
        
    if not os.path.exists(os.path.join(out_dir, "img.nii.gz")):
        dest_img_file = os.path.join(out_dir, "img.nii.gz")
        shutil.copy(src_img_file, dest_img_file)
    
    img_file = os.path.join(out_dir, "img.nii.gz")
    brainMask_file = os.path.join(out_dir,"cache","freesurfer","brainMask.nii.gz")
    brainMaskedImg_file = os.path.join(out_dir, "cache","freesurfer","brainMaskedImg.nii.gz")

    if (not os.path.exists(brainMask_file)) or (not os.path.exists(brainMaskedImg_file)):
        
        subprocess.run(f"mri_synthstrip -i {img_file} -m {brainMask_file} -o {brainMaskedImg_file}", shell=True)

    
    tissueMask_prefix = os.path.join(out_dir, "cache", case, "tissue")
    subprocess.run(f"fast -t 1 -o {tissueMask_prefix} -n 3 {brainMaskedImg_file}", shell=True)
    
    src_mask_file = os.path.join(out_dir, "cache", case, "tissue_seg.nii.gz")
    dest_mask_file = os.path.join(out_dir, f"{case}.nii.gz")
    
    shutil.copy(src_mask_file, dest_mask_file)
        
    
    elapsed_time = time.time()-start_time
    
    df["pid"].append(pid)
    df["case"].append(case)
    df["desc"].append(desc)
    df["time"].append(elapsed_time)
    
    


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [04:07<00:00, 123.61s/it]


# Case-3: Freesurfer (Brain Extraction) + Antspynet (Tissue Segmentation)

In [9]:
case = "case3"
desc = r"Antspynet (Preprocessing) + Freesurfer (Brain Extraction) + Antspynet (Tissue Segmentation)"

In [10]:
for src_img_file in tqdm(src_img_files):
    
    start_time = time.time()
    
    pid = src_img_file.split("/")[-2]
    out_dir = os.path.join(OUT_DIR, pid)
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    if not os.path.exists(os.path.join(out_dir, "cache", case)):
        os.makedirs(os.path.join(out_dir, "cache", case))
        
    if not os.path.exists(os.path.join(out_dir, "img.nii.gz")):
        dest_img_file = os.path.join(out_dir, "img.nii.gz")
        shutil.copy(src_img_file, dest_img_file)
    
    img_file = os.path.join(out_dir, "img.nii.gz")
    
    img = ants.image_read(img_file)
    preprocessed_results = antspynet.preprocess_brain_image(img,truncate_intensity=(0.01, 0.99), brain_extraction_modality="t1", template="croppedMni152", template_transform_type="antsRegistrationSyNQuickRepro[a]", do_bias_correction=True, do_denoising=True)
    
    processedImg = preprocessed_results["preprocessed_image"]#only the preprocessed image without the brain extraction mask applied
    # ants_brainMask_processedImg = preprocessed_results["brain_mask"] #generated by ants
    
    processedImg_file = os.path.join(out_dir, "cache", case, "processedImg.nii.gz")
    # ants_brainMask_processedImg_file = os.path.join(out_dir, "cache", case, "ants_brainMask_processedImg.nii.gz")
    
    ants.image_write(processedImg, processedImg_file)
    # ants.image_write(ants_brainMask_processedImg, ants_brainMask_processedImg_file) #not really needed
        
    
    fs_brainMask_processedImg_file = os.path.join(out_dir,"cache",case,"fs_brainMask_processedImg.nii.gz")
    subprocess.run(f"mri_synthstrip -i {processedImg_file} -m {fs_brainMask_processedImg_file}", shell=True)
        
    fs_brainMask_processedImg = ants.image_read(fs_brainMask_processedImg_file)
    masked_processedImg = processedImg * fs_brainMask_processedImg
    
    results = antspynet.deep_atropos(masked_processedImg, do_preprocessing=False)

    tissueSeg = ants.apply_transforms(fixed=img, moving=results["segmentation_image"], transformlist=preprocessed_results["template_transforms"]["invtransforms"], whichtoinvert=[True], interpolator="nearestNeighbor")

    
    out_mask_file = os.path.join(out_dir, f"{case}.nii.gz")
    ants.image_write(tissueSeg, out_mask_file)
        
    
    elapsed_time = time.time()-start_time
    
    df["pid"].append(pid)
    df["case"].append(case)
    df["desc"].append(desc)
    df["time"].append(elapsed_time)
    
    


  0%|                                                                                                                           | 0/2 [00:00<?, ?it/s]

Preprocessing:  truncate intensities ( low = 4.0 , high = 1446.0 ).
Preprocessing:  brain extraction.
Brain extraction:  retrieving model weights.
5683832/5683832 [==============================] - 1s 0us/step
Brain extraction:  retrieving template.
14969865/14969865 [==============================] - 1s 0us/step


2024-10-10 14:20:58.382903: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: UNKNOWN ERROR (34)
2024-10-10 14:20:58.382951: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fn04.eos.medooza.lan): /proc/driver/nvidia/version does not exist
2024-10-10 14:20:58.383641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Brain extraction:  normalizing image to the template.
Brain extraction:  prediction and decoding.
1/1 [==============================] - 2s 2s/step
Brain extraction:  renormalize probability mask to native space.
19147654/19147654 [==============================] - 1s 0us/step
Brain extraction:  retrieving model weights.
Brain extraction:  retrieving template.
Brain extraction:  normalizing image to the template.
Brain extraction:  prediction and decoding.
1/1 [==============================] - 1s 1s/step
Brain extraction:  renormalize probability mask to native space.
antsRegistration -d 3 -r [0x20bc5470,0x209e8ef0,1] -o [/tmp/tmp49emv38k,0x1efb1bd0,0x1d4dde20] --transform Rigid[0.1] --metric GC[0x20bc5470,0x209e8ef0,1,1,Regular,0.25] --convergence [1000x500x250x0,1e-6,10] --shrink-factors 8x4x2x1 --smoothing-sigmas 3x2x1x0vox --transform Affine[0.1] --metric GC[0x20bc5470,0x209e8ef0,1,1,Regular,0.25] --convergence [1000x500x250x0,1e-6,10] --shrink-factors 8x4x2x1 --smoothing-sigmas 3

OSTIC,    39, -6.061726212502e-01, 3.308588247819e-06, 3.1192e+00, 5.7310e-02, 
 2DIAGNOSTIC,    40, -6.061775088310e-01, 2.972971742565e-06, 3.1730e+00, 5.3822e-02, 
 2DIAGNOSTIC,    41, -6.061827540398e-01, 2.535032308515e-06, 3.2233e+00, 5.0251e-02, 
 2DIAGNOSTIC,    42, -6.061835885048e-01, 2.148514795408e-06, 3.2731e+00, 4.9857e-02, 
 2DIAGNOSTIC,    43, -6.061860918999e-01, 1.834716272242e-06, 3.3245e+00, 5.1393e-02, 
 2DIAGNOSTIC,    44, -6.061850786209e-01, 1.603647433512e-06, 3.3814e+00, 5.6921e-02, 
 2DIAGNOSTIC,    45, -6.061853170395e-01, 1.471517975915e-06, 3.4308e+00, 4.9346e-02, 
 2DIAGNOSTIC,    46, -6.061871051788e-01, 1.272975623579e-06, 3.4866e+00, 5.5815e-02, 
 2DIAGNOSTIC,    47, -6.061843037605e-01, 1.046379225045e-06, 3.5488e+00, 6.2238e-02, 
DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 2DIAGNOSTIC,     1, -5.411189198494e-01, inf, 4.2184e+00, 6.6959e-01, 
  Elapsed time (stage 1): 4.2545e+00


Total elapsed time: 6.0132e+00


...

 50%|█████████████████████████████████████████████████████████▌                                                         | 1/2 [01:33<01:33, 93.59s/it]

Preprocessing:  truncate intensities ( low = 5.0 , high = 2068.0 ).
Preprocessing:  brain extraction.
Brain extraction:  retrieving model weights.
Brain extraction:  retrieving template.
Brain extraction:  normalizing image to the template.
Brain extraction:  prediction and decoding.
1/1 [==============================] - 1s 1s/step
Brain extraction:  renormalize probability mask to native space.
Brain extraction:  retrieving model weights.
Brain extraction:  retrieving template.
Brain extraction:  normalizing image to the template.
Brain extraction:  prediction and decoding.
1/1 [==============================] - 1s 1s/step
Brain extraction:  renormalize probability mask to native space.
antsRegistration -d 3 -r [0x2ab7d9e0,0x2057f470,1] -o [/tmp/tmp68aij8ss,0x2e9623f0,0x20e4b910] --transform Rigid[0.1] --metric GC[0x2ab7d9e0,0x2057f470,1,1,Regular,0.25] --convergence [1000x500x250x0,1e-6,10] --shrink-factors 8x4x2x1 --smoothing-sigmas 3x2x1x0vox --transform Affine[0.1] --metric GC[0x

Using single precision for computations.
Input scalar image: 0x20479f20
Preprocessing:  brain correction.
Reference image: 0x2ab3d580
The composite transform comprises the following transforms (in order): 
  1. /tmp/tmp68aij8ss0GenericAffine.mat (type = AffineTransform)
Default pixel value: 0.0000e+00
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: 0x2f0b4d30
Preprocessing:  denoising.

Running N4 for 3-dimensional images.

Original intensity range:  [5.3321e+00, 2.0680e+03]
Current level = 1
  Iteration 1 (of 50).   Current convergence value = 7.1476e-05 (threshold = 1.0000e-07)
  Iteration 2 (of 50).   Current convergence value = 7.1195e-05 (threshold = 1.0000e-07)
  Iteration 3 (of 50).   Current convergence value = 7.1004e-05 (threshold = 1.0000e-07)
  Iteration 4 (of 50).   Current convergence value = 7.0745e-05 (threshold = 1.0000e-07)
  Iteration 5 (of 50).   Current convergence value = 7.0518e-05 (threshold = 1.0000e-07)
  Iteration 6 (of 50).

Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: outputs/paz9/cache/case3/processedImg.nii.gz
Processing frame (of 1): 1 done
Binary brain mask saved to: outputs/paz9/cache/case3/fs_brainMask_processedImg.nii.gz

If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann
NeuroImage 206 (2022), 119474
https://doi.org/10.1016/j.neuroimage.2022.119474 

Website: https://w3id.org/synthstrip



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:23<00:00, 71.84s/it]


# Case-4: Antspynet (Brain Extraction + Tissue Segmentation)

In [11]:
case = "case4"
desc = r"Antspynet (Brain Extraction + Tissue Segmentation)"

In [12]:
for src_img_file in tqdm(src_img_files):
    
    start_time = time.time()
    
    pid = src_img_file.split("/")[-2]
    out_dir = os.path.join(OUT_DIR, pid)
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    if not os.path.exists(os.path.join(out_dir, "img.nii.gz")):
        dest_img_file = os.path.join(out_dir, "img.nii.gz")
        shutil.copy(src_img_file, dest_img_file)
    
    img_file = os.path.join(out_dir, "img.nii.gz")
    
    img = ants.image_read(img_file)
    
    results = antspynet.deep_atropos(img) #also perform preprocessing
    
    out_mask_file = os.path.join(out_dir, f"{case}.nii.gz")
    ants.image_write(results["segmentation_image"],out_mask_file) 
    
    elapsed_time = time.time()-start_time
    
    df["pid"].append(pid)
    df["case"].append(case)
    df["desc"].append(desc)
    df["time"].append(elapsed_time)
    
    


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:12<00:00, 36.10s/it]


In [13]:
df = pd.DataFrame.from_dict(df)

df.to_csv(os.path.join(OUT_DIR, "stat_detailed.csv"), index=False)

In [ ]:
#stat_summary

In [14]:
summary_df = df.groupby(["case", "desc"], as_index=False).mean()

summary_df.time = summary_df.time/60

In [15]:
summary_df = summary_df.rename(columns={"time":"avg_time (minutes)", "desc":"case_desc"})

In [16]:
summary_df

,case,case_desc,avg_time (minutes)
0,case1,Freesurfer (Brain Extraction) + ANTsX (Tissue ...,2.389257
1,case2,Freesurfer (Brain Extraction) + FSL-FAST (Tiss...,2.060128
2,case3,Antspynet (Preprocessing) + Freesurfer (Brain ...,1.197228
3,case4,Antspynet (Brain Extraction + Tissue Segmentat...,0.601701


In [17]:
summary_df.to_csv(os.path.join(OUT_DIR, "stat_simplified.csv"), index=False)

In [ ]:
# # Case-X: Freesurfer (Brain Extraction + Tissue Segmentation)

# new_df = {"pid":[], "time":[]}

# i = 1

# for src_img_file in tqdm(src_img_files):
    
#     start_time = time.time()
    
#     pid = src_img_file.split("/")[-2]
#     out_dir = os.path.join(OUT_DIR, pid)
    
#     if not os.path.exists(out_dir):
#         os.makedirs(out_dir)
        
#     if not os.path.exists(os.path.join(out_dir, "img.nii.gz")):
#         dest_img_file = os.path.join(out_dir, "img.nii.gz")
#         shutil.copy(src_img_file, dest_img_file)
    
#     img_file = os.path.join(out_dir, "img.nii.gz")
    
#     subprocess.run(f"recon-all -all -subjid subid_{i} -i {img_file}", shell=True)
    
#     elapsed_time = time.time()-start_time
    
#     new_df["pid"].append(pid)
#     new_df["time"].append(elapsed_time)
    
#     i += 1
    
    
# new_df.to_csv("freesurfer.csv", index=False)